In [2]:
import pandas as pd
import geopandas as gpd
import fiona
import osmnx as ox

AttributeError: module 'shapely' has no attribute 'geos_version'

# Etapa 1 : Obtener datos .kml de la ciudad de La Plata.

In [3]:
gdf = gpd.read_file("laplata_cascourbano.kml", driver="KML")
print(gdf.head())


NameError: name 'gpd' is not defined

In [ ]:
# Chequear cuántas capas tiene el archivo kml
for layer in fiona.listlayers("laplata_cascourbano.kml"):
    print(layer)

In [ ]:
# Convertir el archivo kml a geojson
gdf.to_file("salida.geojson", driver='GeoJSON')

In [ ]:
# Plotear el archivo geojson
gdf = gpd.read_file("salida.geojson")
gdf.plot()

# Etapa 2 : Obtener datos de densidad poblacional de la ciudad de La Plata.

In [ ]:
# Dataset de densidad poblacional Argentina 2020 (1km resolución, ~4M puntos)
pd.read_csv("arg_pd_2020_1km_ASCII_XYZ.csv")

In [ ]:
# Función para extraer datos de una ciudad específica del dataset completo
def extract_city_data(city_name: str, bounds: dict) -> pd.DataFrame:

    df = pd.read_csv("arg_pd_2020_1km_ASCII_XYZ.csv")
    
    city_data = df[
        (df['X'] >= bounds['x_min']) & 
        (df['X'] <= bounds['x_max']) & 
        (df['Y'] >= bounds['y_min']) & 
        (df['Y'] <= bounds['y_max'])
    ]
    
    # Guardar subset
    output_file = f"{city_name.lower().replace(' ', '_')}_population_2020.csv"
    city_data.to_csv(output_file, index=False)
    
    print("Filas extraídas:", f"{len(city_data):,}")
    
    return city_data

In [ ]:
# Coordenadas aproximadas para La Plata
la_plata_bounds = {
   'x_min': -58.2,
   'x_max': -57.7,
   'y_min': -35.2,
   'y_max': -34.7
}

# Extraer datos
la_plata_data = extract_city_data("La Plata", la_plata_bounds)


# Etapa 3 : Obtener datos catastrales/domiciliares de la ciudad de La Plata.

In [4]:
# Descargar edificios y direcciones
tags = {'building': True, 'addr:housenumber': True}
gdf = ox.geometries_from_place("La Plata, Buenos Aires, Argentina", tags)

# Revisar resultados
gdf.head()

NameError: name 'ox' is not defined